In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
100% 1.21G/1.21G [00:06<00:00, 260MB/s]
100% 1.21G/1.21G [00:06<00:00, 196MB/s]


In [ ]:
!unzip -q '*.zip'

In [1]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger

# === Config ===
DATA_DIR = "/content/drive/MyDrive/Data/Data/genres_original"
CLASSES = ['blues', 'classical', 'country', 'disco', 'hiphop',
           'jazz', 'metal', 'pop', 'reggae', 'rock']
TARGET_SHAPE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30

# === Checkpoint Setup ===
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = "checkpoints/genre_model_epoch_{epoch:02d}_valacc_{val_accuracy:.2f}.h5"

checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max'
)

csv_logger = CSVLogger('training_log.csv', append=True)

# === GPU Info ===
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# === Data Loader ===
def load_and_preprocess_data(data_dir, classes, target_shape=(150, 150)):
    data = []
    labels = []

    for i_class, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        print("Processing --", class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                try:
                    audio_data, sample_rate = librosa.load(file_path, sr=None)
                except Exception as e:
                    print(f"Skipped {file_path} due to error: {e}")
                    continue

                chunk_duration = 4
                overlap_duration = 2
                chunk_samples = chunk_duration * sample_rate
                overlap_samples = overlap_duration * sample_rate
                num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1

                for i in range(num_chunks):
                    start = i * (chunk_samples - overlap_samples)
                    end = start + chunk_samples
                    chunk = audio_data[start:end]
                    if len(chunk) < chunk_samples:
                        continue
                    mel = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
                    mel = librosa.power_to_db(mel, ref=np.max)
                    mel = np.expand_dims(mel, axis=-1)
                    mel = tf.image.resize(mel, target_shape).numpy()
                    data.append(mel)
                    labels.append(i_class)

    return np.array(data), np.array(labels)

# === Load Data ===
data, labels = load_and_preprocess_data(DATA_DIR, CLASSES, TARGET_SHAPE)
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

labels = to_categorical(labels, num_classes=len(CLASSES))
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# === Load Last Checkpoint if Available ===
latest_model = None
saved_models = sorted([f for f in os.listdir(checkpoint_dir) if f.endswith(".keras")])
if saved_models:
    latest_model = os.path.join(checkpoint_dir, saved_models[-1])
    print(f"Loading model from checkpoint: {latest_model}")
    model = load_model(latest_model)
else:
    # === Build Model from Scratch ===
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=X_train[0].shape))
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))

    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))

    model.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))

    model.add(Conv2D(512, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(512, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(1200, activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(len(CLASSES), activation='softmax'))

# === Compile Model ===
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# === Train Model ===
model.summary()
history = model.fit(
    X_train, Y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, Y_test),
    callbacks=[checkpoint_cb, csv_logger]
)

# === Final Save ===
model.save("genre_classifier_model.keras")
model.save("genre_classifier_model.h5")


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Processing -- blues
Processing -- classical
Processing -- country
Processing -- disco
Processing -- hiphop
Processing -- jazz


/tmp/ipython-input-1-1680560666.py:53: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Skipped /content/drive/MyDrive/Data/Data/genres_original/jazz/jazz.00054.wav due to error: 
Processing -- metal
Processing -- pop
Processing -- reggae
Processing -- rock
Data shape: (17694, 150, 150, 1)
Labels shape: (17694,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

...

443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9251 - loss: 0.2200 - val_accuracy: 0.9234 - val_loss: 0.2335
Epoch 11/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9363 - loss: 0.1819
Epoch 11: val_accuracy did not improve from 0.92342
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9363 - loss: 0.1819 - val_accuracy: 0.9039 - val_loss: 0.2810
Epoch 12/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9494 - loss: 0.1454
Epoch 12: val_accuracy did not improve from 0.92342
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9495 - loss: 0.1454 - val_accuracy: 0.9169 - val_loss: 0.2648
Epoch 13/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9523 - loss: 0.1349
Epoch 13: val_accuracy did not improve from 0.92342
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9523 - loss: 0.1349 - val_accuracy: 0.9005 - val_loss: 0.2996
Epoch 14/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9575 - loss: 0.1148
Epoch 14: val_ac

443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9671 - loss: 0.0925 - val_accuracy: 0.9376 - val_loss: 0.2102
Epoch 16/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9643 - loss: 0.1069
Epoch 16: val_accuracy did not improve from 0.93755
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9643 - loss: 0.1069 - val_accuracy: 0.9333 - val_loss: 0.2091
Epoch 17/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9727 - loss: 0.0751
Epoch 17: val_accuracy did not improve from 0.93755
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9727 - loss: 0.0752 - val_accuracy: 0.9116 - val_loss: 0.2601
Epoch 18/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9802 - loss: 0.0598
Epoch 18: val_accuracy improved from 0.93755 to 0.94716, saving model to checkpoints/genre_model_epoch_18_valacc_0.95.h5


443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9802 - loss: 0.0598 - val_accuracy: 0.9472 - val_loss: 0.1847
Epoch 19/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9795 - loss: 0.0607
Epoch 19: val_accuracy improved from 0.94716 to 0.95535, saving model to checkpoints/genre_model_epoch_19_valacc_0.96.h5


443/443 ━━━━━━━━━━━━━━━━━━━━ 36s 81ms/step - accuracy: 0.9795 - loss: 0.0607 - val_accuracy: 0.9554 - val_loss: 0.1508
Epoch 20/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9842 - loss: 0.0420
Epoch 20: val_accuracy did not improve from 0.95535
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9842 - loss: 0.0421 - val_accuracy: 0.9373 - val_loss: 0.2047
Epoch 21/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9856 - loss: 0.0459
Epoch 21: val_accuracy did not improve from 0.95535
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9856 - loss: 0.0460 - val_accuracy: 0.9291 - val_loss: 0.2462
Epoch 22/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9802 - loss: 0.0567
Epoch 22: val_accuracy did not improve from 0.95535
443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9802 - loss: 0.0567 - val_accuracy: 0.9452 - val_loss: 0.1704
Epoch 23/30
442/443 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9864 - loss: 0.0448
Epoch 23: val_ac

443/443 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - accuracy: 0.9894 - loss: 0.0335 - val_accuracy: 0.9599 - val_loss: 0.1439


In [2]:
import os
import numpy as np
import librosa
import tensorflow as tf
from keras.models import load_model
from collections import Counter

# === Configuration ===
MODEL_PATH = "genre_classifier_model.h5"
AUDIO_PATH = "/content/drive/MyDrive/Data/Taylor Swift - Haunted.mp3"
CLASSES = ['blues', 'classical', 'country', 'disco', 'hiphop',
           'jazz', 'metal', 'pop', 'reggae', 'rock']
TARGET_SHAPE = (150, 150)

# === Load model ===
model = load_model(MODEL_PATH, compile=False)
print("Model loaded.")

def predict_genre_multiple_chunks(audio_path):
    audio_data, sample_rate = librosa.load(audio_path, sr=None)
    chunk_duration = 4
    overlap = 2
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap * sample_rate
    num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1

    predictions = []

    for i in range(num_chunks):
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples
        if end > len(audio_data):
            break
        chunk = audio_data[start:end]

        mel = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
        mel = librosa.power_to_db(mel, ref=np.max)
        mel = np.expand_dims(mel, axis=-1)
        mel = tf.image.resize(mel, TARGET_SHAPE).numpy()
        mel = np.expand_dims(mel, axis=0)

        pred = model.predict(mel, verbose=0)
        class_idx = np.argmax(pred[0])
        predictions.append(class_idx)

    if not predictions:
        print("No valid chunks to predict.")
        return

    vote_counts = Counter(predictions)
    most_common = vote_counts.most_common(1)[0]
    predicted_label = CLASSES[most_common[0]]
    print(f"Predicted Genre: {predicted_label} (from {num_chunks} chunks)")
    print("Votes:", {CLASSES[i]: count for i, count in vote_counts.items()})

# === Run ===
predict_genre_multiple_chunks(AUDIO_PATH)


Model loaded.
Predicted Genre: pop (from 108 chunks)
Votes: {'pop': 98, 'hiphop': 3, 'country': 4, 'rock': 2}
